In [1]:
import numpy as np
import pandas as pd
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
#from sankey import plot_sankey
init_notebook_mode()

In [ ]:
def sankey_df(f, kl, num):
    k=list(zip(kl, kl[1:]))

    df1 = pd.DataFrame([], columns=['source', 'target', 'row_count'])
    if num == None:
      for i in range(len(k)):
        df2=f.groupby(list(k[i]), as_index=False).size()
        df2.columns = ['source', 'target', 'row_count']
        df2['row_count'] = df2['row_count'].astype(int)
        df1 = df1.append(df2).reset_index(drop=True)
      return df1     
    else:
      for i in range(len(k)-1):
        df2=f.groupby(list(k[i]), as_index=False)[num[0]].sum()
        df2.columns = ['source', 'target', 'row_count']
        df1 = df1.append(df2).reset_index(drop=True)
      return df1

def plot_sankey(f, cat, num=None, title=None):
    
    
    if num == None:
      kl = cat
    else:
      num = [num]
      kl = cat + num


    ff = sankey_df(f, kl, num)
    
    colorPalette = ['#001219', '#005f73', '#0a9396', '#94d2bd',
                    '#e9d8a6', '#ee9b00', '#ca6702', '#bb3e03',
                    '#ae2012', '#9b2226']

    labelList = np.union1d(ff['source'].unique(), ff['target'].unique()).tolist()
    colorNumList = []
    
    for catCol in cat:
        labelListTemp =  f[catCol].unique().tolist()
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
    

    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum


    sourceTargetDf = ff.copy()

    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    if title == None:
      title = 'Sankey Diagram : Categorical Variables = ' + str(cat) + ', Numerical Column = ' + str(num)

    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['row_count']
        )
      )

    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
    fig = dict(data=[data], layout=layout)
    py.plot(fig, validate=False)
#     return fig

In [3]:
tp = pd.read_csv('Kangae_new.csv', iterator=True, chunksize=100000)
df = pd.concat(tp, ignore_index=True) 
num_columns = df._get_numeric_data().columns
#numcol = num_columns[1] # Specify numerical column to use as thickness of bands in Sankey
#numcol = 'Var1' # Note: if numcol=None group count will be used

cat_col = [field for field in df.columns if field not in num_columns]  # Specify categorical columns to split data for Sankey nodes
cat_col=['Manufacturer','Product', 'Application']
Title = None # change None to your preffered title otherwise title will be based on cat and num values
if 'numcol' not in locals():
    df['countvals'] = np.ones(df.shape[0])
    numcol = 'countvals'
plot_sankey(df, cat_col, numcol)

KeyError: 'Product'